# Import des bibliothèques

In [ ]:
# Classique
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime
import os
import seaborn as sns
import tqdm
import sys
import string
!{sys.executable} -m pip install spacy

# NLP
!pip install -U spacy
!pip install -U spacy-lookups-data
import spacy
import re
import gensim
!pip install emoji
import emoji
!{sys.executable} -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

!pip install nltk
import nltk
nltk.download('all')
from nltk.stem import PorterStemmer

!pip install lazypredict

# Dataviz textuelle
from wordcloud import WordCloud
from PIL import Image
!pip install geotext
from geotext import GeoText
!pip install better_profanity
from better_profanity import profanity
from textblob import TextBlob 

# ML
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Exploration de modèles
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score 

# Enregistrement du modèle
!pip install joblib
import joblib

# Masquer les messages d'alerte
import warnings
warnings.filterwarnings('ignore')

# Chargement des données

In [ ]:
# le dataset est stocké dans un repo github afin d'avoir un lien dur sur la base
url1 = 'https://raw.githubusercontent.com/Francois-lenne/Big-data-SIAD/main/data/train.csv'
url2 = 'https://raw.githubusercontent.com/Francois-lenne/Big-data-SIAD/main/data/test.csv'

# lecture (depuis github)
train = pd.read_csv('train.csv', sep=',', encoding='utf-8')
test = pd.read_csv('test.csv', sep=',', encoding='utf-8')

# lecture (si stocké dans l'environnement de travail)
train = pd.read_csv(url1, sep=',', encoding='utf-8')
test = pd.read_csv(url2, sep=',', encoding='utf-8')

# Extraction d'informations

In [ ]:
# Nombre de mots
def countWords(text):
  nb_words = len(text.split())
  return nb_words

# Taille moyenne d'un mot
def avgWordLength(text):
  avg = np.mean([len(word) for word in text.split()])
  return avg

# Création de variables binaires indiquant le type de lieu mentionné
def precision(text):
    rep = '0'
    reg1 = r'(,)'
    p = re.compile(reg1)
    check = re.search(reg1,text)
    if check is not None:
      rep = '1'
    return rep

def city(tweet):
    tweet.capitalize()
    places = GeoText(tweet)
    if len(places.cities) == 0:
        cities = 0
    else:
        cities = 1
    return cities

def country(tweet):
    tweet.capitalize()
    places = GeoText(tweet)
    if len(places.countries) == 0:
        countries = 0
    else:
        countries = 1
    return countries

# Détecteur de vulgarités
def isProfanity(text): 
  from better_profanity import profanity 
  check = profanity.contains_profanity(text)
  if check == "false":
      profanity = 0
  else:
      profanity = 1
  return(profanity)

# Récupération des hashtag
def recupHashtag(text):
    """
    Fonction prenant en entrée une chaîne de caractère et retournant une chaîne contenant les hashtags séparés par un espace (s'ils existent).
    """
    reg = r"#[^# ]"
    p = re.compile(reg)
    check = re.search(reg,text)
    if check is not None:
        ge = p.findall(text)
        ge_join = ' '.join(ge)
        return ge_join

# Variable binaire : présence ou non d'un hashtag
def recupHashtagBinaire(text):
    """
    Fonction prenant en entrée une chaîne de caractère et retournant "0" si aucun hashtag n'est présent et "1" si un hashtag est présent.
    """
    rep = '0'
    reg = r"#[^# ]*"
    p = re.compile(reg)
    check = re.search(reg,text)
    if check is not None:
      rep = '1'
    return rep

# Décompte des hashtags
def countHashtag(text):
    """
    Fonction prenant en entrée une chaîne de caractère et retournant retournant le nombre de hashtag # dans le tweet.
    """
    rep = 0
    reg = '#[^# ]*'
    if re.search(reg,text) is not None:
      rep = len(re.findall(reg,text))
    return rep

# Détecte la présence de mentions
def recupNameBinaire(text):
    rep = '0'
    reg = r"@[^@ ]*"
    p = re.compile(reg)
    check = re.search(reg,text)
    if check is not None:
        rep = '1'
    return rep

# Récupération des mentions (@)
def recupName(text):
    """
    Fonction prenant en entrée une chaîne de caractère et retournant une chaîne contenant les mentions séparées par un espace (si elles existent).
    """
    reg = r"@[^@ ]*"
    p = re.compile(reg)
    check = re.search(reg,text)
    if check is not None:
        ge = p.findall(text)
        ge_join = ' '.join(ge)
        return ge_join

# Compte le nombre de mentions (@)
def countName(text):
    """
    Fonction prenant en entrée une chaîne de caractère et retournant le nombre de mentions @ dans le tweet.
    """
    rep = 0
    reg='@[^@ ]*'
    if re.search('@[^@ ]*',text) is not None:
      rep = len(re.findall(reg,text))
    return rep

# Récupération des dates
def recupDate(text):
    """
    Fonction prenant en entrée une chaîne de caractère et retournant une chaîne contenant une date (si elle existe).
    """
    reg = r"([A-Za-z]{3})\s(\d{1,2}),\s(\d{4})"
    p = re.compile(reg)
    check = re.search(reg,text)
    if check is not None:
        ge = p.findall(text)
        return ge

# Variable binaire : présence d'une date
def recupDateBinaire(text):
    """
    Fonction prenant en entrée une chaîne de caractère et retournant "0" si aucune date n'est présente et "1" si une date est présente.
    """
    rep = '0'
    reg = r"([A-Za-z]{3})\s(\d{1,2}),\s(\d{4})"
    p = re.compile(reg)
    check = re.search(reg,text)
    if check is not None:
      rep = '1'
    return rep

# Récupération des liens
def getChemin(text):
    """
    Fonction prenant en entrée une chaîne de caractère et retournant une chaîne contenant le chemin du lien (s'il existe).
    """
    reg1 = r'(https?:\/\/[^\s]+)'
    reg2 = r'(https?)://([^:/]+)(?::(\d+))?(/[^?]*)?(\?[^#]*)?(#.*)?'
    p = re.compile(reg1)
    check = re.search(reg1,text)
    if check is not None:
        ge = p.findall(text)
        for val in ge:
            match = re.search(reg2,val)
            if match:
                rt = match.group(4)
            return rt

# Variable binaire : présence ou non d'un lien
def getCheminBinaire(text):
    """
    Fonction prenant en entrée une chaîne de caractère et retournant "0" si aucun hashtag n'est présent et "1" si un hashtag est présent.
    """
    rep = '0'
    reg1 = r'(https?:\/\/[^\s]+)'
    p = re.compile(reg1)
    check = re.search(reg1,text)
    if check is not None:
      rep = '1'
    return rep

# Variable binaire : présence ou non d'un lien
def countChemin(text):
    """
    Fonction prenant en entrée une chaîne de caractère et retournant "0" si aucun hashtag n'est présent et "1" si un hashtag est présent.
    """
    rep = 0
    reg = '(https?:\/\/[^\s]+)'
    if re.search(reg,text) is not None:
      rep = len(re.findall(reg,text))
    return rep

# Récupération des lieux cités
def getLocation(text):
    """
    Fonction prenant en entrée une chaîne de caractère et retournant une liste de lieu(x) (s'il(s) existe(nt)).
    """
    global nlp
    save = []
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "GPE":
            save.append(ent.text)
    return save

# Variable binaire : présence ou non d'un lieu
def getLocationBinaire(text):
    """
    Fonction prenant en entrée une chaîne de caractère et retournant "0" si aucun hashtag n'est présent et "1" si un hashtag est présent.
    """
    rep = "0"
    global nlp
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "GPE":
            rep = "1"
    return rep

def getSubjectivity(text):
    subj = TextBlob(text).sentiment.subjectivity
    if subj < 0:
        score = 2
    elif subj == 0:
        score = 0
    else:
        score = 1
    return score
    
def getPolarity(text):
    polar = TextBlob(text).sentiment.polarity
    if polar < 0:
        score = 2
    elif polar == 0:
        score = 0
    else:
        score = 1
    return score

def removeElements(text):
  text = re.sub('([A-Za-z]{3})\s(\d{1,2}),\s(\d{4})','',text)
  text = re.sub('@[^@ ]*','',text)
  text = re.sub('#[^# ]','',text)
  text = re.sub('(https?:\/\/[^\s]+)','',text)
  text = re.sub('(https?)://([^:/]+)(?::(\d+))?(/[^?]*)?(\?[^#]*)?(#.*)?','',text)
  return text

def countEmojis(text):
  rep = 0
  reg = '\U0001F600-\U0001F64F|\U0001F300-\U0001F5FF|\U0001F680-\U0001F6FF|\U0001F1E0-\U0001F1FF|\U00002702-\U000027B0|\U000024C2-\U0001F251'
  if re.search(reg,text) is not None:
    rep = len(re.findall(reg,text))
  return rep

In [ ]:
# Conversion en chaîne de caractère - certaines valeurs ne sont en effet pas reconnues comme telles.
train['location'] = train['location'].fillna('')
train['keyword'] = train['keyword'].fillna('')

test['location'] = test['location'].fillna('').replace()
test['keyword'] = test['keyword'].fillna('')

# Nombre de caractères du tweet, espaces inclus
train['text_length'] = [len(text) for text in train['text']]
test['text_length'] = [len(text) for text in test['text']]

# Catégorie de taille des tweets
train['tweet_length_cut'] = pd.cut(train['text_length'], [0,100,140], labels=['<100','>100'])
test['tweet_length_cut'] = pd.cut(test['text_length'], [0,100,140], labels=['<100','>100'])

# Nombre de mots
train['nb_mots'] = train['text'].apply(countWords)
test['nb_mots'] = test['text'].apply(countWords)

# Longueur des mots
train['longueur_mots'] = train['text'].apply(avgWordLength)
test['longueur_mots'] = test['text'].apply(avgWordLength)


# Présence ou non d'une ville
train['city'] = train['location'].apply(city)
test['city'] = test['location'].apply(city)

# Présence ou non d'un pays
train['country'] = train['location'].apply(country)
test['country'] = test['location'].apply(country)

# présence d'un lieu ou non
train['precision'] = train['location'].apply(precision)
test['precision'] = test['location'].apply(precision)

# Présence ou non d'une vulgarité
# train['profanity'] = train['text'].apply(isProfanity)
# test['profanity'] = test['text'].apply(isProfanity)

# Compte des emojis
train['nb_emojis'] = train['text'].apply(countEmojis)
test['nb_emojis'] = test['text'].apply(countEmojis)

# Extraction des hashtags
train['hashtags'] = train['text'].apply(recupHashtag)
test['hashtags'] = test['text'].apply(recupHashtag)

# Présence d'un hashtag
train['hashtags_b'] = train['text'].apply(recupHashtagBinaire)
test['hashtags_b'] = test['text'].apply(recupHashtagBinaire)

# Décompte des hashtags
train['nb_hashtags'] = train['text'].apply(countHashtag)
test['nb_hashtags'] = test['text'].apply(countHashtag)

# Extraction des mentions
train['mentions'] = train['text'].apply(recupName)
test['mentions'] = test['text'].apply(recupName)

# Présence d'une mention
train['mentions_b'] = train['text'].apply(recupNameBinaire)
test['mentions_b'] = test['text'].apply(recupNameBinaire)

# Décompte des mentions
train['nb_mentions'] = train['text'].apply(countName)
test['nb_mentions'] = test['text'].apply(countName)

# Extraction des dates
train['dates'] = train['text'].apply(recupDate)
test['dates'] = test['text'].apply(recupDate)

# Présence d'une date
train['dates_b'] = train['text'].apply(recupDateBinaire)
test['dates_b'] = test['text'].apply(recupDateBinaire)

# Extraction des liens
train['rt_path'] = train['text'].apply(getChemin)
test['rt_path'] = test['text'].apply(getChemin)

# Présence d'un lien
train['rt_path_b'] = train['text'].apply(getCheminBinaire)
test['rt_path_b'] = test['text'].apply(getCheminBinaire)

# Décompte des liens
train['nb_liens'] = train['text'].apply(countChemin)
test['nb_liens'] = test['text'].apply(countChemin)

# Analyse de sentiment
## Subjectivité
train['Subjectivity'] = train['text'].apply(getSubjectivity)
test['Subjectivity'] = test['text'].apply(getSubjectivity)

## Polarité
train['Polarity'] = train['text'].apply(getPolarity)
test['Polarity'] = test['text'].apply(getPolarity)

# Présence ou non d'une ville
train['text'] = train['text'].apply(removeElements)
test['text'] = test['text'].apply(removeElements)

# Nettoyage

In [ ]:
def preprocessing(text):
    """
    Fonction prenant en entrée une chaîne de caractères et retournant cette chaîne de caractères
    après avoir appliqué l'ensemble des modifications décrites ci-dessous.
    """
    text = str(text)

    # Harmonisation de la casse : mise en minuscule
    text = text.lower()

    # retrait espaces entre les nombres
    text = re.sub(r'(?<=\\d) +(?=\\d)', '', text) 

    # Gestion des accents et ponctuations
    text = re.sub(r'%20',' ', text) # remplacement %20 par un espace
    text = re.sub(r'&amp|& amp', '&', text) # remplacement &amp par &
    # text = re.sub("\d+", " ", text) # retrait nombres
    text = re.sub('[éèê]', "e", text) # retrait accents sur le e
    text = re.sub("[ôöóò]", "o", text) # retrait accents sur le o
    text = re.sub("[üùû]", "u", text) # retrait accents sur le u
    text = re.sub("[ïiî]", "i", text) # retrait accents sur le i
    text = re.sub("[âàäå]", "a", text) # retrait accents sur le a
    text = re.sub("[_.,;:!?]", " ", text) # retrait ponctuation
    text = re.sub("[|{}()«»/]", " ", text) # retrait parenthèses, guillemets, slashs...
    text = re.sub("[“”]", " ", text) # retrait guillemets (autre forme)
    text = re.sub("'", " ", text) # retrait apostrophes
    text = re.sub("’", " ", text) # retrait apostrophes (autre forme)
    text = re.sub('"', " ", text) # retrait quotes
    text = re.sub('[+-]', " ", text) # retrait + et -
    text = re.sub('[=*/]', " ", text) # retrait opérateurs
    text = re.sub("°", "", text) # retrait symbole °

    # Gestion des symboles
    text = re.sub("[€%$£]", "", text) # retrait symboles devises

    # Gestions des retours à la ligne ou fin de lignes (caractères non-imprimables)
    text = re.sub('\r\n', " ", text) # retrait retour charriot/retour à la ligne
    text = re.sub('\n', " ", text) # retrait retour à la ligne

    # Gestion des espaces
    text = re.sub('\s+', " ", text) # retrait espaces en trop
    text = text.rstrip(" ") # retrait espaces à droite
    text = text.lstrip(" ") # retrait espaces à gauche

    # Si un nombre est collé à un texte, les sépare avec un espace
    text = re.sub(r'(?<=[a-zA-Z])(?=\d)|(?<=\d)(?=[a-zA-Z])', ' ',text)

    # Gestion des emojis
    emoji_pattern = re.compile(
    '['
    u'\U0001F600-\U0001F64F'  # emoticons
    u'\U0001F300-\U0001F5FF'  # symbols & pictographs
    u'\U0001F680-\U0001F6FF'  # transport & map symbols
    u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
    u'\U00002702-\U000027B0'
    u'\U000024C2-\U0001F251'
    ']+',
    flags=re.UNICODE)
    emoji_pattern.sub(r'', text)

    return text

In [ ]:
# Application de la fonction de nettoyage
train['text_CLEAN'] = train['text'].apply(preprocessing)
train['keyword'] = train['keyword'].apply(preprocessing)

test['text_CLEAN'] = test['text'].apply(preprocessing)
test['keyword'] = test['keyword'].apply(preprocessing)

# Analyse exploratoire

In [ ]:
# df = train.groupby(['target'])['nb_liens'].mean().reset_index()
# sns.barplot(data=df, x="target", y="nb_liens").set(title = 'Nombre de liens', xticklabels = ['fake','non-fake'])

In [ ]:
# Diagramme en barres
# pd.crosstab(train['target'],train['country'],normalize='index').plot(kind='bar').set(title='mention d\'un pays',xticklabels = ['fake','non-fake'])

In [ ]:
# # Diagramme en barres
# graph = sns.countplot(y ='location',
#                       hue = 'target',
#                       data=train,
#                       order=train.location.value_counts().iloc[1:16].index)
# graph.set(title = 'Top 15 localisations')
# graph.legend(labels=['fake','non-fake'],
#            loc='upper center',
#            bbox_to_anchor=(1.2, 1))

# Traitements divers NLP

In [ ]:
#########################################
# LEMMISATION OU STEMMISATION + STOPWORDS
#########################################

nlp = spacy.load('en_core_web_sm')
stopwords = nltk.corpus.stopwords.words('english')

def Lemmatization(train,texts):
  pbar = tqdm.tqdm(total=train.shape[0])
  nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
  texts_out = []
  for text in texts:
      doc = nlp(text)
      new_text = []
      for token in doc: 
          new_text.append(token.lemma_)
      final = " ".join(new_text)
      texts_out.append(final)
      pbar.update(1) 
  return (texts_out)
  pbar.close()

train['text_CLEAN_LMT'] = Lemmatization(train,train['text_CLEAN'])
test['text_CLEAN_LMT'] = Lemmatization(test,test['text_CLEAN'])

def Remove_stopwords(text):
  text = [word for word in text.split() if word not in stopwords and len(word)>2]
  return text

train['text_CLEAN_LMT_TOKEN_WSW'] = train['text_CLEAN_LMT'].apply(Remove_stopwords)
test['text_CLEAN_LMT_TOKEN_WSW'] = test['text_CLEAN_LMT'].apply(Remove_stopwords)

########################
# BIGRAMMES - TRIGRAMMES
########################

# Fonctions utilitaires
def make_bigrams(texts):
    return [bigram_mod[text] for text in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[text]] for text in texts]

# Train
data_words_train = train['text_CLEAN_LMT_TOKEN_WSW'].values.tolist()
data_words_train

bigram = gensim.models.Phrases(data_words_train, threshold = 1, scoring = 'npmi')
trigram = gensim.models.Phrases(bigram[data_words_train], threshold = 1, scoring = 'npmi')

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

data_words_train_bigrams = make_bigrams(data_words_train)
train['text_CLEAN_LMT_TOKEN_WSW_BIGRAMS'] = data_words_train_bigrams

# Test
data_words_test = test['text_CLEAN_LMT_TOKEN_WSW'].values.tolist()
data_words_test

bigram = gensim.models.Phrases(data_words_test, min_count=5, threshold=1, scoring = 'npmi')
trigram = gensim.models.Phrases(bigram[data_words_test], threshold=1, scoring = 'npmi')

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

data_words_test_bigrams = make_bigrams(data_words_test)
test['text_CLEAN_LMT_TOKEN_WSW_BIGRAMS'] = data_words_test_bigrams

#################
# DETOKENIZATION
#################

def Detokenize(txt):
    txt = ' '.join([word for word in txt])
    return txt

train['text_CLEAN_LMT_WSW_BIGRAMS'] = train['text_CLEAN_LMT_TOKEN_WSW_BIGRAMS'].apply(lambda x: Detokenize(x))
test['text_CLEAN_LMT_WSW_BIGRAMS'] = test['text_CLEAN_LMT_TOKEN_WSW_BIGRAMS'].apply(lambda x: Detokenize(x))

# Nuages de mots FAKE vs NOT FAKE

In [ ]:
twitter_mask = np.array(Image.open('twitter_logo.jpg'))

# Création d'une table dont les colonnes sont les mots et les données sont les valeurs TF-Itrain
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(train['text_CLEAN_LMT_WSW_BIGRAMS'].loc[train['target']==0].astype('U'))
feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
df = pd.DataFrame(dense, columns=feature_names)

wordcloud = WordCloud(background_color='white', mask = twitter_mask, colormap = 'winter')
wordcloud.generate_from_frequencies(df.T.sum(axis=1))

plt.figure(figsize=(15,15))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# Création d'une table dont les colonnes sont les mots et les données sont les valeurs TF-Itrain
vectors = vectorizer.fit_transform(train['text_CLEAN_LMT_WSW_BIGRAMS'].loc[train['target']==1].astype('U'))
feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
df = pd.DataFrame(dense, columns=feature_names)

wordcloud = WordCloud(background_color='white', mask = twitter_mask, colormap = 'winter')
wordcloud.generate_from_frequencies(df.T.sum(axis=1))

plt.figure(figsize=(15,15))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# Transformations des variables

In [ ]:
# Vectorisation

print('Vectorisation des tweets d\'apprentissage en cours ...')
# Création d'une table dont les colonnes sont les mots et les données sont les valeurs TF-Itrain
text_vectorizer = TfidfVectorizer()
# Entraîne le modèle sur les données d'entraînement (on s'assure que les données textuelles sont bien encodée en unicode)
text_vectorizer_fitted = text_vectorizer.fit(train['text_CLEAN_LMT_WSW_BIGRAMS'].astype('U'))
# On enregistre ce modèle pour le réutiliser dans l'API (pour transformer les tweets passés en entrée)
joblib.dump(text_vectorizer_fitted,'tweet_vectorizer.gz')
# Applique la transformation
text_vectors = text_vectorizer_fitted.transform(train['text_CLEAN_LMT_WSW_BIGRAMS'].astype('U'))
# Récupère la liste des mots distincts qui deviennent des noms de features
text_feature_names = text_vectorizer_fitted.get_feature_names_out()
# Structure les données sous la forme d'un dataframe où 1 colonne = 1 mot de la liste de tous les mots présents dans les tweets
text_dense = text_vectors.todense()
train_tfidf = pd.DataFrame(text_dense, columns=text_feature_names)
print('Vectorisation des tweets d\'apprentissage terminée.')


print('Vectorisation des tweets de test en cours ...')
# Applique la même transformation sur les données de test que sur les données d'apprentissage.
# Cela permet d'obtenir une table structurée avec les mêmes mots-colonnes.
text_vectors = text_vectorizer_fitted.transform(test['text_CLEAN_LMT_WSW_BIGRAMS'].astype('U'))
# Récupère la liste des mots distincts qui deviennent des noms de features
text_feature_names = text_vectorizer_fitted.get_feature_names_out()
# Structure les données sous la forme d'un dataframe où 1 colonne = 1 mot de la liste de tous les mots présents dans les tweets
text_dense = text_vectors.todense()
test_tfidf = pd.DataFrame(text_dense, columns=text_feature_names)
print('Vectorisation des tweets de test terminée.')

In [ ]:
train_tfidf.sample(5)

# Sélection des variables

In [ ]:
print('Sélection des variables en cours ...')

# Données d'entraînement
X = train_tfidf # mots (devenus des variables)
y = train['target'] # cible

# Fonction initialisant un modèle de sélection des variables basée sur le chi2 entre chaque variable explicatives et la cible 2 à 2.
# Ici on exclut les X% des variables les moins associées à la cible, donc les X% les moins discriminantes.
# Cette sélection permet de réduire la dimensionnalité du modèle et par conséquent, de limiter le surapprentissage.
chi2_features = SelectKBest(chi2, k = int(X.shape[1]*0.9))
# Ajustement du modèle sur les données d'apprentissage. On crée ainsi la liste des mots à conserver dans les tweets.
chi2_features_selector = chi2_features.fit(X,y)
# On enregistre ce modèle pour le réutiliser dans l'API (pour transformer les tweets passés en entrée)
joblib.dump(chi2_features_selector,'features_selector.gz')
# On applique le modèle sur les tweets d'apprentissage 
features_selected = chi2_features_selector.transform(X)
features_names = chi2_features_selector.get_feature_names_out()
# On stocke la sortie dans une table
train_features = pd.DataFrame(features_selected,columns=features_names)

# Données de test
X = test_tfidf 
# On applique le modèle (précédemment ajusté sur les données d'entraînements) sur les tweets d'apprentissage 
features_selected = chi2_features_selector.transform(X)
features_names = chi2_features_selector.get_feature_names_out()
# On stocke la sortie dans une table
test_features = pd.DataFrame(features_selected,columns=features_names)

print('Sélection des variables terminée.')

In [ ]:
joblib.dump(chi2_features_selector,'features_selector.gz')

# Modélisation

In [ ]:
train_variables_quali = pd.get_dummies(train[['keyword','text_length']])
train_variables_quanti = train[['nb_mots','nb_hashtags','nb_mentions','nb_liens','longueur_mots','city','country','precision']]

final_train = pd.concat([train_features, # variables-mots créées à l'aide des tweets (features TF IDF)
                         train_variables_quanti, # variables quantitatives créées via l'extraction de données dans les tweets
                         train_variables_quali], # variables qualitatives créées via l'extraction de données dans les tweets
                         axis = 'columns')

X_train = final_train.reset_index(drop=True)
y_train = train['target']

# Exploration des modèles

In [ ]:
# # Echantillons de 30% des jeux de données
# X_train_sample = X_train.sample(frac=0.3)
# y_train_sample = y_train.sample(frac=0.3)

# pipe = Pipeline([
#     ("scaler", StandardScaler()), # mean = True ne fonctionne pas avec une sparse matrix (TD-IDF)
#     # Modèle testé en premier (n'importe lequel)
#     ("classifier", LogisticRegression(random_state=0))
# ])

# param_grid = [

#     # Régression Logistique (one vs rest)
#     # {'classifier': [LogisticRegression(random_state=0)],
#     #  'classifier__penalty':['l1','l2'],
#     #  },

#     # Support Vector Machine
#     {'classifier': [SVC(kernel='rbf', random_state=0, probability = True)],
#     #  'classifier__kernel' : ['rbf','linear','poly'],
#      'classifier__C':[0.01,0.1,1]
#      },
    
#     # Réseau de neurones
#     # {'classifier': [MLPClassifier(hidden_layer_sizes=(100,))],
#     #  },
# ]

# # Appel de la fonction d'exploration de modèles
# grid = GridSearchCV(pipe, param_grid, verbose = 2, cv = 3) # verbose pour l'affichage du temps de traitement
# # Ajustement des modèles

# grid.fit(X_train_sample, y_train_sample)

# # Visualisation des résultats
# print('best model : ', grid.best_params_)
# print('best cv score : ', grid.best_score_)

# Entrainement et validation

In [ ]:
# Echantillonnage
X_train_t, X_train_v, y_train_t, y_train_v = train_test_split(X_train,
                                                              y_train,
                                                              test_size=0.2)

# Transformation séparée pour éviter la fuite des données
scaler = StandardScaler()
X_train_t = scaler.fit_transform(X_train_t)
X_train_v = scaler.fit_transform(X_train_v)

# Entraînement
SVC_model = SVC(kernel='rbf',probability = True, random_state=0, C = 1)
SVC_model.fit(X_train_t,y_train_t)

# Predictions
y_pred = SVC_model.predict(X_train_v)

# Validation
print('f1 score : ', f1_score(y_train_v,y_pred))
print('accuracy score : ', accuracy_score(y_train_v,y_pred))

# Entraînement (sur tout l'échantillon) et enregistrement

In [ ]:
# SVM
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Chargement du modèle retenu
SVC_model = SVC(kernel='rbf',probability = True, random_state=0, C = 1)
# Ajustement du modèle sur l'échantillon total
SVC_model.fit(X_train,y_train)

# Enregistrement du modèle
joblib.dump(SVC_model, 'Model.gz')

# Prédictions sur données de test (pour la soumission)

In [ ]:
# Concaténation de l'ensemble des features du jeu de données de test
test_variables_quali = pd.get_dummies(test[['keyword','text_length']])
test_variables_quanti = test[['nb_mots','nb_hashtags','nb_mentions','nb_liens','longueur_mots','city','country','precision']]

final_test = pd.concat([test_features, # variables-mots créées à l'aide des tweets (features TF IDF)
                        test_variables_quanti, # variables quantitatives créées via l'extraction de données dans les tweets
                        test_variables_quali], # variables qualitatives créées via l'extraction de données dans les tweets
                         axis='columns')

X_test = final_test.reset_index(drop=True)

X_test = scaler.fit_transform(X_test)

test_predictions_tab = pd.DataFrame(SVC_model.predict(X_test)) \
                            .reset_index(drop=True).rename(columns={0 : 'target_prédite'})

test_probas_tab = pd.DataFrame(SVC_model.predict_proba(X_test), columns=[SVC_model.classes_])

test_results = pd.concat([test[['id','text']],
                          test_predictions_tab,
                          test_probas_tab],
                          axis=1).dropna() # probas et prédictions

test_results.dropna().to_csv('test_results.csv',sep=';',encoding='utf-16')

submission = test_results.rename(columns = {'target_prédite' : 'target'})
submission = submission[['id','target']].set_index('id')
submission.to_csv('submission_SVC.csv')